In [2]:
import pandas as pd
import numpy as np
import math
import time
import collections
from datetime import datetime


## af_user_loan_info

In [51]:
#pd_data = pd.read_csv('af_loan_1.csv',engine='python')
#pd_data = pd.read_excel('af_loan_1.xlsx')


#改变列名
pd_data.columns = ['phone_no','certNo','name','loanDate',\
                   'loanTypeCode','orgName','periods','loanAmount',\
                   'approvalStatusCode','loanStatusCode','overdueStatus',\
                   'overdueAmount','overdueTotal','overdueM3','overdueM6','getTime','importTime']

# 时间统一为 年-月
pd_data['loanDate'] = pd_data['loanDate'].map(lambda x:pd.to_datetime(str(x),format='%Y%m.0'))#.map(lambda x: datetime.strftime(x,"%Y-%m")).map(lambda x:pd.to_datetime(str(x),format='%Y-%m'))
pd_data['getTime'] = pd_data['getTime'].map(lambda x:pd.to_datetime(str(x),format='%Y-%m-%d'))#.map(lambda x: datetime.strftime(x,"%Y-%m")).map(lambda x:pd.to_datetime(str(x),format='%Y-%m'))

# 月份差
#pd_data['month_diff'] = pd_data[['getTime','loanDate']].apply(lambda x:(x[0]-x[1]),axis=1).map(lambda x:int(x.days//30))
pd_data['month_diff'] = pd_data[['getTime','loanDate']].apply(lambda x:(x[0]-x[1]),axis=1).map(lambda x:x.days//30)

period = (('30d',1),
          ('90d',3),
          ('180d',6),
          ('360d',12),
          ('all',100000))


# loanAmount取中间值
d={'(0,1000]':500,'(1000,5000]':3000,'(5000,10000]':7500,'(10000,20000]':15000,'(20000,50000]':35000,'(50000,100000]':75000,'(100000,+)':100000}
pd_data['loanAmount'] = pd_data['loanAmount'].map(lambda x:d.get(x))
pd_data['overdueAmount'] = pd_data['overdueAmount'].map(lambda x:d.get(x))


# certNo作为索引
id_data = pd.DataFrame(index=pd_data['certNo'].drop_duplicates().sort_values().values)

In [52]:
for prdname,prdtime in period:
    
    prddata = pd_data[pd_data['month_diff']<=prdtime]
    
    #### PART 1 申请状态
    #### approvalStatusCode 审批结果码 201.审核中 202.批贷已放款 203.拒贷 204.客户放弃

    # n天内申请总次数 
    id_data['af_n_Inquiry_' + prdname] = prddata[['certNo','loanDate']].groupby('certNo').count()
    
    # n天内申请总次数（审批结果码=审核中）
    id_data['af_n_Inquiry_Checking_' + prdname] = prddata[prddata['approvalStatusCode']==201][['certNo','loanDate']].groupby('certNo').count() 
    
    # n天内申请总次数（审批结果码=批贷已放款）
    id_data['af_n_Inquiry_Approved_' + prdname] = prddata[prddata['approvalStatusCode']==202][['certNo','loanDate']].groupby('certNo').count()
    
    # n天内申请总次数（审批结果码=拒贷）
    id_data['af_n_Inquiry_Rejected_' + prdname] = prddata[prddata['approvalStatusCode']==203][['certNo','loanDate']].groupby('certNo').count()
    
    # n天内申请总次数（审批结果码=客户放弃）
    id_data['af_n_Inquiry_Aborted_' + prdname] = prddata[prddata['approvalStatusCode']==204][['certNo','loanDate']].groupby('certNo').count()
    
    # n天内申请次数通过率
    id_data['af_n_Inquiry_Pass_rate_' + prdname] = id_data[['af_n_Inquiry_Approved_' + prdname,'af_n_Inquiry_' + prdname]].apply(lambda x:round(x[0]/x[1],2), axis=1)
    
    
    
    # n天内申请总金额
    id_data['af_sum_Inquiry_' + prdname] = prddata[['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内申请总金额（审批结果码=审核中）
    id_data['af_sum_Inquiry_Checking_' + prdname] = prddata[prddata['approvalStatusCode']==201][['certNo','loanAmount']].groupby('certNo').sum() 
    
    # n天内申请总金额（审批结果码=批贷已放款）
    id_data['af_sum_Inquiry_Approved_' + prdname] = prddata[prddata['approvalStatusCode']==202][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内申请总金额（审批结果码=拒贷）
    id_data['af_sum_Inquiry_Rejected_' + prdname] = prddata[prddata['approvalStatusCode']==203][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内申请总金额（审批结果码=客户放弃）
    id_data['af_sum_Inquiry_Aborted_' + prdname] = prddata[prddata['approvalStatusCode']==204][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内申请金额通过率
    id_data['af_sum_Inquiry_Pass_rate_' + prdname] = id_data[['af_sum_Inquiry_Approved_' + prdname,'af_sum_Inquiry_' + prdname]].apply(lambda x:round(x[0]/x[1],2), axis=1)
    
    
    
    # n天内申请最大金额
    id_data['af_max_Inquiry_' + prdname] = prddata[['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内申请最大金额（审批结果码=审核中）
    id_data['af_max_Inquiry_Checking_' + prdname] = prddata[prddata['approvalStatusCode']==201][['certNo','loanAmount']].groupby('certNo').max() 
    
    # n天内申请最大金额（审批结果码=批贷已放款）
    id_data['af_max_Inquiry_Approved_' + prdname] = prddata[prddata['approvalStatusCode']==202][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内申请最大金额（审批结果码=拒贷）
    id_data['af_max_Inquiry_Rejected_' + prdname] = prddata[prddata['approvalStatusCode']==203][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内申请最大金额（审批结果码=客户放弃）
    id_data['af_max_Inquiry_Aborted_' + prdname] = prddata[prddata['approvalStatusCode']==204][['certNo','loanAmount']].groupby('certNo').max()
    
    
    
    # n天内放款最大期数
    id_data['af_max_Period_' + prdname] = prddata[['certNo','periods']].groupby('certNo').max()

    # n天内放款最小期数
    id_data['af_min_Period_' + prdname] = prddata[['certNo','periods']].groupby('certNo').min()
    
    # n天内放款平均期数
    id_data['af_mean_Period_' + prdname] = prddata[['certNo','periods']].groupby('certNo').mean().apply(lambda x:round(x,2))
    
    
    
    #### PART 2 放款状态
    #### loanStatusCode 还款状态 301.正常 302.逾期 303.结清
    #### overdueStatus 历史逾期情况 M1, M2等
    
    # n天内总放款次数（还款状态=正常）
    id_data['af_n_Repay_Normal_' + prdname] = prddata[prddata['loanStatusCode'] == '301'][['certNo','loanStatusCode']].groupby('certNo').count()
    
    # n天内总放款次数（还款状态=逾期）
    id_data['af_n_Repay_Odue_' + prdname] = prddata[prddata['loanStatusCode'] == '302'][['certNo','loanStatusCode']].groupby('certNo').count()
    
    # n天内总放款次数（还款状态=结清）
    id_data['af_n_Repay_Clear_' + prdname] = prddata[prddata['loanStatusCode'] == '303'][['certNo','loanStatusCode']].groupby('certNo').count()
    
    # n天内总放款次数（当前逾期情况=M1）
    id_data['af_n_Repay_M1_' + prdname] = prddata[prddata['overdueStatus'] == 'M1'][['certNo','overdueStatus']].groupby('certNo').count()
    
    # n天内总放款次数（当前逾期情况=M1及以上）
    id_data['af_n_Repay_M1plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M1','M2','M3','M3+','M4','M5','M6','M6+'])][['certNo','overdueStatus']].groupby('certNo').count()
    
    # n天内总放款次数（当前逾期情况=M2）
    id_data['af_n_Repay_M2_' + prdname] = prddata[prddata['overdueStatus'] == 'M2'][['certNo','overdueStatus']].groupby('certNo').count()
    
    # n天内总放款次数（当前逾期情况=M2及以上）
    id_data['af_n_Repay_M2plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M2','M3','M3+','M4','M5','M6','M6+'])][['certNo','overdueStatus']].groupby('certNo').count()
    
    # n天内总放款次数（当前逾期情况=M3）
    id_data['af_n_Repay_M3_' + prdname] = prddata[prddata['overdueStatus'] == 'M3'][['certNo','overdueStatus']].groupby('certNo').count()
    
    # n天内总放款次数（当前逾期情况=M3及以上）
    id_data['af_n_Repay_M3plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M3','M3+','M4','M5','M6','M6+'])][['certNo','overdueStatus']].groupby('certNo').count()
    
    
    
    
    # n天内总放款金额（还款状态=正常）
    id_data['af_sum_Repay_Normal_' + prdname] = prddata[prddata['loanStatusCode'] == '301'][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（还款状态=逾期）
    id_data['af_sum_Repay_Odue_' + prdname] = prddata[prddata['loanStatusCode'] == '302'][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（还款状态=结清）
    id_data['af_sum_Repay_Clear_' + prdname] = prddata[prddata['loanStatusCode'] == '303'][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（当前逾期情况=M1）
    id_data['af_sum_Repay_M1_' + prdname] = prddata[prddata['overdueStatus'] == 'M1'][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（当前逾期情况=M1及以上）
    id_data['af_sum_Repay_M1plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M1','M2','M3','M3+','M4','M5','M6','M6+'])][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（当前逾期情况=M2）
    id_data['af_sum_Repay_M2_' + prdname] = prddata[prddata['overdueStatus'] == 'M2'][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（当前逾期情况=M2及以上）
    id_data['af_sum_Repay_M2plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M2','M3','M3+','M4','M5','M6','M6+'])][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（当前逾期情况=M3）
    id_data['af_sum_Repay_M3_' + prdname] = prddata[prddata['overdueStatus'] == 'M3'][['certNo','loanAmount']].groupby('certNo').sum()
    
    # n天内总放款金额（当前逾期情况=M3及以上）
    id_data['af_sum_Repay_M3plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M3','M3+','M4','M5','M6','M6+'])][['certNo','loanAmount']].groupby('certNo').sum()
    
    
    
    
    # n天内最大放款金额（还款状态=正常）
    id_data['af_max_Repay_Normal_' + prdname] = prddata[prddata['loanStatusCode'] == '301'][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（还款状态=逾期）
    id_data['af_max_Repay_Odue_' + prdname] = prddata[prddata['loanStatusCode'] == '302'][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（还款状态=结清）
    id_data['af_max_Repay_Clear_' + prdname] = prddata[prddata['loanStatusCode'] == '303'][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（当前逾期情况=M1）
    id_data['af_max_Repay_M1_' + prdname] = prddata[prddata['overdueStatus'] == 'M1'][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（当前逾期情况=M1及以上）
    id_data['af_max_Repay_M1plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M1','M2','M3','M3+','M4','M5','M6','M6+'])][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（当前逾期情况=M2）
    id_data['af_max_Repay_M2_' + prdname] = prddata[prddata['overdueStatus'] == 'M2'][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（当前逾期情况=M2及以上）
    id_data['af_max_Repay_M2plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M2','M3','M3+','M4','M5','M6','M6+'])][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（当前逾期情况=M3）
    id_data['af_max_Repay_M3_' + prdname] = prddata[prddata['overdueStatus'] == 'M3'][['certNo','loanAmount']].groupby('certNo').max()
    
    # n天内最大放款金额（当前逾期情况=M3及以上）
    id_data['af_max_Repay_M3plus_' + prdname] = prddata[prddata['overdueStatus'].isin(['M3','M3+','M4','M5','M6','M6+'])][['certNo','loanAmount']].groupby('certNo').max()
    
    
    
    
    #### PART 3 申请类型
    #### loanTypeCode 借款类型码 21.信用 22.抵押 23.担保
    
    # n天内总申请次数（申请类型=信用）
    id_data['af_n_Type_Credit_' + prdname] = prddata[prddata['loanTypeCode'] == 21][['certNo','loanTypeCode']].groupby('certNo').count()
    
    # n天内总申请次数（申请类型=抵押）
    id_data['af_n_Type_Deposit_' + prdname] = prddata[prddata['loanTypeCode'] == 22][['certNo','loanTypeCode']].groupby('certNo').count()
    
    # n天内总申请次数（申请类型=担保）
    id_data['af_n_Type_Assure_' + prdname] = prddata[prddata['loanTypeCode'] == 23][['certNo','loanTypeCode']].groupby('certNo').count()

    



In [53]:
#### PART 5 时间相关

# 最晚一次申请距今月数
id_data['af_min_Inquiry_date_gap'] = pd_data[['certNo','month_diff']].groupby('certNo').min()

# 最早一次申请距今月数
id_data['af_max_Inquiry_date_gap'] = pd_data[['certNo','month_diff']].groupby('certNo').max()

# 最晚一次逾期距今月数
id_data['af_min_Odue_date_gap'] = pd_data[pd_data['overdueTotal']>0][['certNo','month_diff']].groupby('certNo').min()

# 最早一次逾期距今月数
id_data['af_max_Odue_date_gap'] = pd_data[pd_data['overdueTotal']>0][['certNo','month_diff']].groupby('certNo').max()

####### 逾期判断  overdueTotal


In [54]:
id_data.fillna(0,inplace=True)

id_data = id_data.reset_index().rename(columns={'index':'certNo'})

id_data = pd.DataFrame(id_data)
id_data.to_excel('af_loan_1_character.xlsx',index=False)

## af_user_query_info

In [ ]:
pd_data_q = pd.read_csv('af_user_query_info_from_json_test.csv',engine='python')

#pd_data_q = pd.read_excel('af_query_1.xlsx')

id_data_q = pd.DataFrame(index=pd_data_q['certNo'].drop_duplicates().sort_values().values)

# 时间统一为 年-月
pd_data_q['time'] = pd_data_q['time'].map(lambda x:pd.to_datetime(str(x),format='%Y-%m-%d'))#.map(lambda x: datetime.strftime(x,"%Y-%m")).map(lambda x:pd.to_datetime(str(x),format='%Y-%m'))
pd_data_q['getTime'] = pd_data_q['getTime'].map(lambda x:pd.to_datetime(str(x),format='%Y%m%d%H')).map(lambda x: datetime.strftime(x,"%Y-%m-%d")).map(lambda x:pd.to_datetime(str(x),format='%Y-%m'))

# 月份差
pd_data_q['month_diff'] = pd_data_q[['getTime','time']].apply(lambda x:(x[0]-x[1]),axis=1).map(lambda x:int(x.days//30))

for prdname,prdtime in period:
    
    prddata = pd_data_q[pd_data_q['month_diff']<=prdtime]
    
    #### PART 4 查询相关
    #### orgType 机构类型
    #### queryReason 查询原因：10贷款审批，11贷后管理，12信用卡审批，13担保资格审查，14保前审查
    
    # n天内总查询次数（机构类型包含"P2P"）
    id_data_q['af_n_Query_P2P_' + prdname] = prddata[prddata['orgType'].str.contains('P2P')][['certNo','orgType']].groupby('certNo').count()

    # n天内总查询次数（机构类型包含"持牌消费金融"）
    id_data_q['af_n_Query_LicConFin_' + prdname] = prddata[prddata['orgType'].str.contains('持牌消费金融')][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"非持牌消金"）
    id_data_q['af_n_Query_NoLicConFin_' + prdname] = prddata[prddata['orgType'].str.contains('非持牌消金')][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"资金平台"）
    id_data_q['af_n_Query_FundPlat_' + prdname] = prddata[prddata['orgType'].str.contains('资金平台')][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"融资租赁"）
    id_data_q['af_n_Query_Rent_' + prdname] = prddata[prddata['orgType'].str.contains('融资租赁')][['certNo','orgType']].groupby('certNo').count()
    
    
    # n天内总查询次数（机构类型包含"保险"）
    id_data_q['af_n_Query_Safe_' + prdname] = prddata[prddata['orgType'].str.contains('保险')][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"'典当','担保','银行','平台门户'"）
    id_data_q['af_n_Query_Other_' + prdname] = prddata[prddata['orgType'].str.contains('典当')|prddata['orgType'].str.contains('平台门户')|prddata['orgType'].str.contains('银行')|prddata['orgType'].str.contains('担保')][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"小贷"）
    id_data_q['af_n_Query_Small_' + prdname] = prddata[prddata['orgType'].str.contains('小额现金贷')|prddata['orgType'].str.contains('小额信贷')|prddata['orgType'].str.contains('小贷')][['certNo','orgType']].groupby('certNo').count()
    

    
    # n天内总查询次数（查询原因=贷款审批）
    id_data_q['af_n_Query_LoanCheck_' + prdname] = prddata[prddata['queryReason'] == 10][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=贷后管理）
    id_data_q['af_n_Query_LoanManage_' + prdname] = prddata[prddata['queryReason'] == 11][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=信用卡审批）
    id_data_q['af_n_Query_CardCheck_' + prdname] = prddata[prddata['queryReason'] == 12][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=担保资格审查）
    id_data_q['af_n_Query_AssureQualCheck_' + prdname] = prddata[prddata['queryReason'] == 13][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=保前审查）
    id_data_q['af_n_Query_AssureCheck_' + prdname] = prddata[prddata['queryReason'] == 14][['certNo','queryReason']].groupby('certNo').count()
    
    
    
id_data_q.fillna(0,inplace=True)

id_data_q = id_data_q.reset_index().rename(columns={'index':'certNo'})

id_data_q = pd.DataFrame(id_data_q)
id_data_q.to_excel('af_query_1_character.xlsx',index=False)

### 不分日期版

In [48]:
pd_data_q = pd.read_excel('af_query_1.xlsx')

#修改列名
pd_data_q.columns = ['phone_no','flow_id','certNo','orgName','orgType','queryReason']

id_data_q = pd.DataFrame(index=pd_data_q['certNo'].drop_duplicates().sort_values().values)

    #### PART 4 查询相关
    #### orgType 机构类型
    #### queryReason 查询原因：10贷款审批，11贷后管理，12信用卡审批，13担保资格审查，14保前审查
    
    # n天内总查询次数（机构类型包含"P2P"）
id_data_q['af_n_Query_P2P_all'] = pd_data_q[pd_data_q['orgType'].str.contains('P2P', na=False)][['certNo','orgType']].groupby('certNo').count()

    # n天内总查询次数（机构类型包含"持牌消费金融"）
id_data_q['af_n_Query_LicConFin_all'] = pd_data_q[pd_data_q['orgType'].str.contains('持牌消费金融', na=False)][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"非持牌消金"）
id_data_q['af_n_Query_NoLicConFin_all'] = pd_data_q[pd_data_q['orgType'].str.contains('非持牌消金', na=False)][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"资金平台"）
id_data_q['af_n_Query_FundPlat_all'] = pd_data_q[pd_data_q['orgType'].str.contains('资金平台', na=False)][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"融资租赁"）
id_data_q['af_n_Query_Rent_all'] = pd_data_q[pd_data_q['orgType'].str.contains('融资租赁', na=False)][['certNo','orgType']].groupby('certNo').count()
    
    
    # n天内总查询次数（机构类型包含"保险"）
id_data_q['af_n_Query_Safe_all'] = pd_data_q[pd_data_q['orgType'].str.contains('保险', na=False)][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"'典当','担保','银行','平台门户'"）
id_data_q['af_n_Query_Other_all'] = pd_data_q[pd_data_q['orgType'].str.contains('典当', na=False)|pd_data_q['orgType'].str.contains('平台门户', na=False)|pd_data_q['orgType'].str.contains('银行', na=False)|pd_data_q['orgType'].str.contains('担保', na=False)][['certNo','orgType']].groupby('certNo').count()
    
    # n天内总查询次数（机构类型包含"小贷"）
id_data_q['af_n_Query_Small_all'] = pd_data_q[pd_data_q['orgType'].str.contains('小额现金贷', na=False)|pd_data_q['orgType'].str.contains('小额信贷', na=False)|pd_data_q['orgType'].str.contains('小贷', na=False)][['certNo','orgType']].groupby('certNo').count()
    

    
    # n天内总查询次数（查询原因=贷款审批）
id_data_q['af_n_Query_LoanCheck_all'] = pd_data_q[pd_data_q['queryReason'] == 10][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=贷后管理）
id_data_q['af_n_Query_LoanManage_all'] = pd_data_q[pd_data_q['queryReason'] == 11][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=信用卡审批）
id_data_q['af_n_Query_CardCheck_all'] = pd_data_q[pd_data_q['queryReason'] == 12][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=担保资格审查）
id_data_q['af_n_Query_AssureQualCheck_all'] = pd_data_q[pd_data_q['queryReason'] == 13][['certNo','queryReason']].groupby('certNo').count()
    
    # n天内总查询次数（查询原因=保前审查）
id_data_q['af_n_Query_AssureCheck_all'] = pd_data_q[pd_data_q['queryReason'] == 14][['certNo','queryReason']].groupby('certNo').count()
    
    
    
id_data_q.fillna(0,inplace=True)

id_data_q = id_data_q.reset_index().rename(columns={'index':'certNo'})

id_data_q = pd.DataFrame(id_data_q)
id_data_q.to_excel('af_query_1_character.xlsx',index=False)